This script imputes Temporary Assistance for Needy Families (TANF) recipients dollar benefit amount to match the aggregates with United States Department of Health & Human Services (HHS) statistics for TANF. In this current version, we used 2015 March CPS data and HHS FY2014 caseload and expenditures data on TANF. Please refer to the documentation in the same folder for more details on methodology and assumptions. The output this script is an individual level dataset that contains CPS personal level ID (PERIDNUM), individual participation indicator (tanf_participation, 0 - not a recipient, 1 - current recipient on file, 2 - imputed recipient), and benefit amount.

Input: 2015 CPS (asec2015_pubuse.csv), number of recipients and their benefits amount by state in 2014 (TANF_administrative.csv)

Output: TANF_Imputation.csv

Additional Source links: https://www.acf.hhs.gov/ofa/programs/tanf/data-reports 

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

## Variables used in TANF

In [2]:
CPS_dataset = pd.read_csv('asec2015_pubuse.csv')
columns_to_keep = ['paw_val','paw_typ','paw_mon','fpawval','marsupwt','a_age','a_sex','wsal_val','semp_val','frse_val',
                  'ss_val','rtm_val','oi_val','oi_off','int_yn','uc_yn', 'uc_val','int_val','ssi_yn','ssikidyn',
                  'hfoodsp','a_famnum','a_maritl','fownu6','fownu18','gestfips','peridnum']
CPS_dataset = CPS_dataset[columns_to_keep]
CPS_dataset.to_csv('TANF.csv', columns=columns_to_keep, index=False)

/Users/intern/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,22,23,28,80,187,273) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
CPS_dataset = pd.read_csv('TANF.csv')

In [4]:
# TANF value
tanfvalue = pd.to_numeric(np.where(CPS_dataset.paw_typ== 'TANF/AFDC', CPS_dataset.paw_val, 0))
# Also include small part of other Public Assistance
tanfvalue = pd.to_numeric(np.where(CPS_dataset.paw_typ== 'Both', CPS_dataset.paw_val, tanfvalue))
# TANF indicator
indicator = pd.to_numeric(np.where(CPS_dataset.paw_typ== 'TANF/AFDC', 1, 0))
indicator = pd.to_numeric(np.where(CPS_dataset.paw_typ== 'Both', 1, indicator))

In [5]:
TANF = DataFrame(tanfvalue.transpose())
TANF.columns = ['tanfvalue']

In [6]:
TANF['indicator'] = indicator
TANF['marsupwt'] = CPS_dataset.marsupwt
TANF['tanfvalue'] = tanfvalue
TANF['gestfips'] = CPS_dataset.gestfips
TANF['peridnum'] = CPS_dataset.peridnum

In [7]:
# Number of month receiving TANF
month = np.where(CPS_dataset.paw_mon == 'Not in universe', 0, CPS_dataset.paw_mon)
month = np.where(CPS_dataset.paw_mon == 'Twelve', 12, month)
month = np.where(CPS_dataset.paw_mon == 'One', 1, month)
month = pd.to_numeric(month)
TANF['month'] = month

In [8]:
# Earned income
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
earned = wage + self_employed1 + self_employed2 #individual earned income
TANF['earned'] = earned

In [9]:
# Unearned income
ss = pd.to_numeric(np.where(CPS_dataset.ss_val!='None or not in universe', CPS_dataset.ss_val, 0))
pension = pd.to_numeric(np.where(CPS_dataset.rtm_val!='None or not in universe', CPS_dataset.rtm_val, 0))
disability = pd.to_numeric(np.where(CPS_dataset.oi_off=='State disability payments', CPS_dataset.oi_val, 0))
unemploy = pd.to_numeric(np.where(CPS_dataset.uc_yn=='Yes', CPS_dataset.uc_val, 0))
interest = pd.to_numeric(np.where(CPS_dataset.int_yn=='Yes', CPS_dataset.int_val, 0))
unearned = ss + pension + disability + unemploy + interest #individual unearned income
TANF['unearned'] = unearned

In [10]:
# Net Income
TANF['net_income'] = earned + unearned

In [11]:
# Family TANF value
familyvalue = np.where(CPS_dataset.fpawval != 'None', CPS_dataset.fpawval,0)
TANF['familyvalue'] = familyvalue
familyindicator = np.where(TANF['familyvalue'] != 0, 1,0)
familyotherTANFindicator = familyindicator - indicator
TANF['indicatorOther'] = familyotherTANFindicator                               

In [12]:
# Prepare age information
age = np.where(CPS_dataset.a_age == "80-84 years of age",
                             random.randrange(80, 84),
                             CPS_dataset.a_age)
age = np.where(CPS_dataset.a_age == "85+ years of age",
                             random.randrange(85, 95),
                             age)
TANF['a_age'] = pd.to_numeric(age)

In [13]:
# Prepare gender inforamtion, 0 for male, 1 for female
TANF['sex'] = pd.to_numeric(np.where(CPS_dataset.a_sex == 'Male', 0, 1))

In [14]:
# Number of children
childunder6 = np.where(CPS_dataset.fownu6 == 'None, not in universe', 0, CPS_dataset.fownu6)
TANF['childunder6'] = pd.to_numeric(childunder6)
childunder18 = np.where(CPS_dataset.fownu18 == 'None, not in universe', 0, CPS_dataset.fownu18)
childunder18 = np.where(CPS_dataset.fownu18 == '9 or more', 9, childunder18)
TANF['childunder18'] = pd.to_numeric(childunder18)
TANF['child6to18'] = TANF['childunder18'] - TANF['childunder6']

In [15]:
# Status of Marriage
marriage = np.where(CPS_dataset.a_maritl == 'Married - civilian spouse', 1, 0)
marriage = np.where(CPS_dataset.a_maritl == 'Married - AF spouse present', 1, marriage)
marriage = np.where(CPS_dataset.a_maritl == 'Married - spouse absent (exc.', 1, marriage)
marriage = np.where(CPS_dataset.a_maritl == 'Separated', 1, marriage)
TANF['marriage'] = pd.to_numeric(marriage)

In [16]:
TANF['unemploy_indicator'] = np.where(CPS_dataset.uc_yn=='Yes', 1, 0)
TANF['ssi_indicator'] = np.where((CPS_dataset.ssi_yn=='Yes')|(CPS_dataset.ssikidyn=='Received SSI'), 1, 0)
TANF['snap_indicator'] = np.where(CPS_dataset.hfoodsp == "Yes",1,0)

## Regression Model

In [17]:
TANF['intercept'] = np.ones(len(TANF))
model = sm.Logit(endog=TANF.indicator, exog=TANF[['intercept','a_age', 'sex', 
                                                  'childunder6','child6to18', 'earned', 'unearned','indicatorOther', 
                                                  'unemploy_indicator','ssi_indicator','snap_indicator',
                                                  'marriage']]).fit()
print model.summary()

         Current function value: 0.019560
         Iterations: 35


/Users/intern/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                           Logit Regression Results                           
Dep. Variable:              indicator   No. Observations:               199024
Model:                          Logit   Df Residuals:                   199012
Method:                           MLE   Df Model:                           11
Date:                Fri, 04 Aug 2017   Pseudo R-squ.:                  0.2662
Time:                        16:08:35   Log-Likelihood:                -3892.8
converged:                      False   LL-Null:                       -5305.2
                                        LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
intercept             -8.6636      0.158    -54.864      0.000      -8.973      -8.354
a_age                  0.0231      0.002     10.547      0.000       0.019       0.027
sex                 

In [18]:
probs = model.fittedvalues
TANF['probs'] = probs

## Import administrative data

In [19]:
admin = pd.read_csv('TANF_Administrative.csv',
                    dtype = {'Total Annual Benefits': np.float, 'Total Annual Recipient': np.float, 'Average Annual Benefits' : np.float})
admin.index = admin.Fips

In [20]:
# CPS total benefits and Administrative total benefits, annual basis
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (TANF.gestfips==state)
    CPS_totalb = (TANF.tanfvalue * TANF.marsupwt)[this_state].sum()
    admin_totalb =  admin['Total Annual Benefits'][state] 
    CPS_totaln = (TANF.marsupwt[this_state & TANF.indicator==1] * TANF.month / 12).sum()
    admin_totaln =  admin['Total Annual Recipient'][state]

    temp = [admin.State[state], CPS_totalb, admin_totalb, CPS_totaln, admin_totaln]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total benefits','Admin total benefits',
                               'CPS total recipients','Admin total recipients']

In [21]:
pre_augment_benefit.to_csv('pre-blow-up.csv')

## Imputation

In [22]:
# caculate difference of SNAP stats and CPS aggregates on recipients number
# by state
diff = {'Fips':[],'Difference in Population':[],'Mean Benefit':[],'CPS Population':[],'TANF Population':[]}
diff['Fips'] = admin.Fips
current = (TANF.indicator==1)
for FIPS in admin.Fips:
        this_state = (TANF.gestfips==FIPS)
        current_tots = (TANF.marsupwt[current&this_state]*TANF.month).sum()/12
        valid_num = (TANF.marsupwt[current&this_state]*TANF.month).sum()/12 + 0.0000001
        current_mean = ((TANF.tanfvalue * TANF.marsupwt)[current&this_state].sum())/valid_num
        diff['CPS Population'].append(current_tots)
        diff['TANF Population'].append(float(admin['Total Annual Recipient'][admin.Fips == FIPS]))
        diff['Difference in Population'].append(float(admin['Total Annual Recipient'][admin.Fips == FIPS])- current_tots)
        diff['Mean Benefit'].append(current_mean)

In [23]:
d = DataFrame(diff)
d = d[['Fips', 'Mean Benefit', 'Difference in Population', 'CPS Population', 'TANF Population']]
d.index = d.Fips
d.to_csv('recipients.csv', index=False)

In [24]:
TANF['impute'] = np.zeros(len(TANF))
TANF['tanf_impute'] = np.zeros(len(TANF))

non_current = (TANF.indicator==0)
current = (TANF.indicator==1)
random.seed()

for FIPS in admin.Fips:
    
        print ('we need to impute', d['Difference in Population'][FIPS], 'for state', FIPS)
        
        if d['Difference in Population'][FIPS] < 0:
            continue
        else:
            this_state = (TANF.gestfips==FIPS)
            not_imputed = (TANF.impute==0)
            pool_index = TANF[this_state&not_imputed&non_current].index
            pool = DataFrame({'weight': TANF.marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            pool = pool.sort_values(by='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d['Difference in Population'][FIPS])
            min_index = pool.sort_values(by='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            TANF.impute[pool.index[pool['impute']==1]] = 1
            TANF.tanf_impute[pool.index[pool['impute']==1]] = admin['Average Annual Benefits'][FIPS]
           
        print ('Method1: regression gives', 
                TANF.marsupwt[(TANF.impute==1)&this_state].sum())

('we need to impute', 27841.905003333333, 'for state', 1)


/Users/intern/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/intern/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Method1: regression gives', 28420.43)
('we need to impute', 6262.1575003333328, 'for state', 2)
('Method1: regression gives', 6161.149999999999)
('we need to impute', 21909.155003333333, 'for state', 4)
('Method1: regression gives', 21613.53)
('we need to impute', 9437.1158366666659, 'for state', 5)
('Method1: regression gives', 9539.070000000002)
('we need to impute', 1039643.1120000002, 'for state', 6)
('Method1: regression gives', 1039437.4699999997)
('we need to impute', 37973.335830000004, 'for state', 8)
('Method1: regression gives', 39559.10000000001)
('we need to impute', 16362.730003333332, 'for state', 9)
('Method1: regression gives', 16157.160000000002)
('we need to impute', 10512.781663333333, 'for state', 10)
('Method1: regression gives', 10581.630000000001)
('we need to impute', 11508.409169999999, 'for state', 11)
('Method1: regression gives', 11402.189999999999)
('we need to impute', 51636.105836666669, 'for state', 12)
('Method1: regression gives', 51594.63999999999)

In [25]:
#Adjustment ratio
results = {}

imputed = (TANF.impute == 1)
has_val = (TANF.tanfvalue != 0)
no_val = (TANF.tanfvalue == 0)

for FIPS in admin.Fips:
    this_state = (TANF.gestfips==FIPS)
    
    current_total = (TANF.tanfvalue * TANF.marsupwt)[this_state].sum() 
    imputed_total = (TANF.tanf_impute * TANF.marsupwt)[this_state&imputed].sum()
    on_file = current_total + imputed_total

    admin_total = admin['Total Annual Benefits'][FIPS]
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    TANF.tanf_impute = np.where(has_val&this_state, TANF.tanfvalue * adjust_ratio, TANF.tanf_impute)
    TANF.tanf_impute = np.where(no_val&this_state, TANF.tanf_impute * adjust_ratio, TANF.tanf_impute)

TANF["tanf_participation"] = np.zeros(len(TANF))
TANF["tanf_participation"] = np.where(TANF.impute==1, 2, 0)#Augmented
TANF["tanf_participation"] = np.where(has_val, 1, TANF.tanf_participation)#CPS 


r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
r.to_csv('amount.csv', index=False)

In [26]:
TANF.to_csv('TANF_Imputation.csv', 
                   columns=['peridnum', 'tanf_participation','tanf_impute', 'marsupwt','month'],
                   index=False)